# RAG Brug af Azure AI Agent Service & Semantic Kernel

Denne kodeeksempel viser, hvordan man opretter og administrerer en Azure AI-agent til retrieval-augmented generation (RAG) ved hjælp af `Azure AI Agent Service` og `Semantic Kernel`. Agenten behandler brugerforespørgsler baseret på den hentede kontekst og giver præcise svar derefter.


SQLite Version Fix  
Hvis du støder på fejlen:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Fjern kommenteringen af denne kodeblok i starten af din notebook:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importering af pakker
Den følgende kode importerer de nødvendige pakker:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Retrieval-Augmented Generation med Semantic Kernel & Azure AI Agent Service

Dette eksempel demonstrerer, hvordan man bruger **Azure AI Agent Service** til at udføre **Retrieval-Augmented Generation (RAG)** ved at kombinere en sprogmodel med domænespecifik kontekst fra et uploadet dokument.

### Sådan fungerer det

1. **Dokumentupload**: En markdown-fil (document.md), der indeholder information (Contosos rejseforsikringspolitik), uploades til agenttjenesten.

2. **Oprettelse af vektorlager**: Dokumentet indekseres i et vektorlager for at muliggøre semantisk søgning i dets indhold.

3. **Agentkonfiguration**: En agent oprettes ved hjælp af `gpt-4o`-modellen med følgende strenge instruktioner:
   - Besvar kun spørgsmål baseret på hentet indhold fra dokumentet.
   - Afvis at besvare, hvis spørgsmålet ligger uden for dokumentets rammer.

4. **Integration af File Search Tool**: `FileSearchTool` registreres hos agenten, hvilket gør det muligt for modellen at søge og hente relevante uddrag fra det indekserede dokument under inferens.

5. **Brugerinteraktion**: Brugere kan stille spørgsmål. Hvis relevant information findes i dokumentet, genererer agenten et velunderbygget svar.  
   Hvis ikke, svarer agenten eksplicit, at dokumentet ikke indeholder tilstrækkelig information.


### Hovedfunktion

```python
def main():
    # Initialiserer programmet
    print("Velkommen til programmet!")

    # Kalder hjælpefunktionen
    helper_function()
```

### Hjælpefunktion

```python
def helper_function():
    # Udfører en simpel opgave
    print("Hjælpefunktionen er blevet kaldt.")
```

### Fejlhåndtering

```python
try:
    # Forsøger at udføre en opgave
    result = perform_task()
except Exception as e:
    # Håndterer fejl
    print(f"En fejl opstod: {e}")
```

### Konklusion

```python
if __name__ == "__main__":
    # Starter hovedfunktionen
    main()
```


Sørg for først at køre `az login` ved hjælp af Azure CLI, så den korrekte autentificeringskontekst leveres, når du bruger `DefaultAzureCredential`. Azure AI Agent Service bruger ikke API-nøgler.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Understood! Please provide the markdown file you'd like me to translate.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at opnå nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
